In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [73]:
# The code was removed by Watson Studio for sharing.

,Town,Latitude,Longitude
0,ANG MO KIO,1.369972,103.849588
1,BEDOK,1.324011,103.930172
2,BISHAN,1.351042,103.849930
3,BUKIT BATOK,1.348506,103.749222
4,BUKIT MERAH,1.289642,103.816798


In [74]:
df_data_0.shape

(25, 3)

In [75]:
df_data_0

,Town,Latitude,Longitude
0,ANG MO KIO,1.369972,103.849588
1,BEDOK,1.324011,103.930172
2,BISHAN,1.351042,103.849930
3,BUKIT BATOK,1.348506,103.749222
4,BUKIT MERAH,1.289642,103.816798
5,BUKIT PANJANG,1.276068,103.791904
6,CENTRAL,1.288155,103.846718
7,CHOA CHU KANG,1.385385,103.744337
8,CLEMENTI,1.315070,103.765246
9,GEYLANG,1.316367,103.882772


In [76]:
address = 'singapore'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


In [6]:
# create map of Toronto using latitude and longitude values
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, town in zip(df_data_0['Latitude'], df_data_0['Longitude'], df_data_0['Town'], ):
    label = '{}'.format(town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_singapore)  
    
map_singapore

In [7]:
CLIENT_ID = 'Q003U2R31HOT5FJQH1YHG40PCC2L3GSKD3QQREXJXQFIPE4Q' # your Foursquare ID
CLIENT_SECRET = '22EQRULXQF1H11PFWBR2RL4CFTX31TVCC5VHGUI5R1RFF04G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q003U2R31HOT5FJQH1YHG40PCC2L3GSKD3QQREXJXQFIPE4Q
CLIENT_SECRET:22EQRULXQF1H11PFWBR2RL4CFTX31TVCC5VHGUI5R1RFF04G


In [10]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,latitude,longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Q003U2R31HOT5FJQH1YHG40PCC2L3GSKD3QQREXJXQFIPE4Q&client_secret=22EQRULXQF1H11PFWBR2RL4CFTX31TVCC5VHGUI5R1RFF04G&ll=1.357107,103.8194992&v=20180605&radius=500&limit=100'

In [26]:
radius = 500
LIMIT = 100

venues = []

for lat, long, town, in zip(df_data_0['Latitude'], df_data_0['Longitude'], df_data_0['Town']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            town,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
venues_df = pd.DataFrame(venues)


# define the column names
venues_df.columns = ['Town', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory',]

venues_df

,Town,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,ANG MO KIO,1.369972,103.849588,FairPrice Xtra,1.369279,103.848886,Supermarket
1,ANG MO KIO,1.369972,103.849588,Old Chang Kee,1.369094,103.848389,Snack Place
2,ANG MO KIO,1.369972,103.849588,MOS Burger,1.369170,103.847831,Burger Joint
3,ANG MO KIO,1.369972,103.849588,Subway,1.369136,103.847612,Sandwich Place
4,ANG MO KIO,1.369972,103.849588,NTUC FairPrice,1.371507,103.847082,Supermarket
5,ANG MO KIO,1.369972,103.849588,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House
6,ANG MO KIO,1.369972,103.849588,Express Teppan-Yaki,1.369137,103.848232,Japanese Restaurant
7,ANG MO KIO,1.369972,103.849588,"Coffee Shop, Block 422, Ang Mo Kio Ave 3",1.368119,103.851093,Coffee Shop
8,ANG MO KIO,1.369972,103.849588,PLAYe,1.369109,103.848225,Hobby Shop
9,ANG MO KIO,1.369972,103.849588,A&W,1.369541,103.849043,Fast Food Restaurant


In [24]:
!pip install --upgrade wget
import wget

!pip install --upgrade findspark
import findspark

!pip install --upgrade pyspark
import pyspark as sc


Requirement already up-to-date: wget in /opt/conda/envs/Python36/lib/python3.6/site-packages (3.2)
Requirement already up-to-date: findspark in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.3.0)
Requirement already up-to-date: pyspark in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.4.5)


In [29]:
#from project_lib import Project
#from pyspark import SparkContext

project = Project(sc,"b3e313a0-3ecf-444c-8247-5074787dc169","p-55ff4d0fffd48facaed365a039fcfe673ab07125")
project.save_data("Shop Location.csv", venues_df.to_csv())


{'file_name': 'Shop Location.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneprojectnotebookibm-donotdelete-pr-lknbpoeyg4qbzh',
 'asset_id': '4a5c7cdf-ffbd-4fb6-a6fe-ecf6efbbbb40'}

,Town,Latitude,Longitude
0,ANG MO KIO,1.369972,103.849588
1,BEDOK,1.324011,103.930172
2,BISHAN,1.351042,103.849930
3,BUKIT BATOK,1.348506,103.749222
4,BUKIT MERAH,1.289642,103.816798


In [115]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Town,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,0,ANG MO KIO,1.369972,103.849588,FairPrice Xtra,1.369279,103.848886,Supermarket
1,1,ANG MO KIO,1.369972,103.849588,Old Chang Kee,1.369094,103.848389,Snack Place
2,2,ANG MO KIO,1.369972,103.849588,MOS Burger,1.369170,103.847831,Burger Joint
3,3,ANG MO KIO,1.369972,103.849588,Subway,1.369136,103.847612,Sandwich Place
4,4,ANG MO KIO,1.369972,103.849588,NTUC FairPrice,1.371507,103.847082,Supermarket


In [97]:
shop = df_data_1.loc[:, ~df_data_1.columns.str.contains('^Unnamed')]
shop

,Town,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,ANG MO KIO,1.369972,103.849588,FairPrice Xtra,1.369279,103.848886,Supermarket
1,ANG MO KIO,1.369972,103.849588,Old Chang Kee,1.369094,103.848389,Snack Place
2,ANG MO KIO,1.369972,103.849588,MOS Burger,1.369170,103.847831,Burger Joint
3,ANG MO KIO,1.369972,103.849588,Subway,1.369136,103.847612,Sandwich Place
4,ANG MO KIO,1.369972,103.849588,NTUC FairPrice,1.371507,103.847082,Supermarket
5,ANG MO KIO,1.369972,103.849588,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House
6,ANG MO KIO,1.369972,103.849588,Express Teppan-Yaki,1.369137,103.848232,Japanese Restaurant
7,ANG MO KIO,1.369972,103.849588,"Coffee Shop, Block 422, Ang Mo Kio Ave 3",1.368119,103.851093,Coffee Shop
8,ANG MO KIO,1.369972,103.849588,PLAYe,1.369109,103.848225,Hobby Shop
9,ANG MO KIO,1.369972,103.849588,A&W,1.369541,103.849043,Fast Food Restaurant


In [16]:
print('There are {} uniques categories.'.format(len(shop['VenueCategory'].unique())))

There are 170 uniques categories.


In [17]:
shop.VenueCategory.unique()

array(['Supermarket', 'Snack Place', 'Burger Joint', 'Sandwich Place',
       'Noodle House', 'Japanese Restaurant', 'Coffee Shop', 'Hobby Shop',
       'Fast Food Restaurant', 'Dessert Shop', 'Bakery',
       'Miscellaneous Shop', 'Food Court', 'Gym', 'Chinese Restaurant',
       'Gym / Fitness Center', 'Bubble Tea Shop', 'Shopping Mall',
       'Modern European Restaurant', 'Bank', 'Multiplex',
       'Ramen Restaurant', 'Pharmacy', 'Seafood Restaurant',
       'Frozen Yogurt Shop', 'Fried Chicken Joint', 'Convenience Store',
       'Sushi Restaurant', 'Malay Restaurant', 'French Restaurant',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot',
       'Hotpot Restaurant', 'Wings Joint', 'Dumpling Restaurant',
       'Sports Bar', 'Burrito Place', 'Asian Restaurant', 'Soup Place',
       'Bookstore', 'Thrift / Vintage Store', 'Ice Cream Shop',
       'Indian Restaurant', 'American Restaurant', 'Clothing Store',
       'Karaoke Bar', 'Café', 'Hong Kong Restaurant',
       'Italian

In [66]:
#shop_res = shop[shop.VenueCategory == 'Restaurant']
#shop_res = shop['VenueCategory'].str.contains("Restaurant")
# shop_res = shop[shop['VenueCategory'].str.contains("Restaurant")] 
# shop_res

In [100]:


shop.loc[shop['VenueCategory'].str.contains('Restaurant', case=False), 'VenueCategory'] = 'Restaurant'
shop

,Town,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,ANG MO KIO,1.369972,103.849588,FairPrice Xtra,1.369279,103.848886,Supermarket
1,ANG MO KIO,1.369972,103.849588,Old Chang Kee,1.369094,103.848389,Snack Place
2,ANG MO KIO,1.369972,103.849588,MOS Burger,1.369170,103.847831,Burger Joint
3,ANG MO KIO,1.369972,103.849588,Subway,1.369136,103.847612,Sandwich Place
4,ANG MO KIO,1.369972,103.849588,NTUC FairPrice,1.371507,103.847082,Supermarket
5,ANG MO KIO,1.369972,103.849588,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House
6,ANG MO KIO,1.369972,103.849588,Express Teppan-Yaki,1.369137,103.848232,Restaurant
7,ANG MO KIO,1.369972,103.849588,"Coffee Shop, Block 422, Ang Mo Kio Ave 3",1.368119,103.851093,Coffee Shop
8,ANG MO KIO,1.369972,103.849588,PLAYe,1.369109,103.848225,Hobby Shop
9,ANG MO KIO,1.369972,103.849588,A&W,1.369541,103.849043,Restaurant


In [101]:
# one hot encoding
kl_onehot = pd.get_dummies(shop[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = shop['Town'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1055, 137)


,Neighborhoods,Accessories Store,Arcade,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Canal,Cha Chaan Teng,Chinese Breakfast Place,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fish & Chips Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Music School,Music Venue,Night Market,Nightclub,Noodle House,Office,Outdoor Supply Store,Outlet Store,Park,Pastry Shop,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Recreation Center,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Video Game Store,Waterfront,Wine Shop,Wings Joint,Yoga Studio
0,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [102]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(25, 137)


,Neighborhoods,Accessories Store,Arcade,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Canal,Cha Chaan Teng,Chinese Breakfast Place,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fish & Chips Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Music School,Music Venue,Night Market,Nightclub,Noodle House,Office,Outdoor Supply Store,Outlet Store,Park,Pastry Shop,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Recreation Center,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Taxi Stand,Tea Room,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Video Game Store,Waterfront,Wine Shop,Wings Joint,Yoga Studio
0,ANG MO KIO,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.025000,0.025,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.025000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.100000,0.025000,0.000000,0.00,0.000000,0.075000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075000,0.000000,0.025000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.025000,0.000000,0.00,0.025,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.025000,0.000000,0.00,0.000000,0.00000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.050000,0.00000,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00
1,BEDOK,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.016667,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.016667,0.000000,0.016667,0.00,0.00,0.016667,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.016667,0.033333,0.00,0.000000,0.000000,0.016667,0.00,0.066667,0.000000,0.000000,0.00,0.000000,0.016667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.016667,0.016667,0.000000,0.016667,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.00,0.000000,0.383333,0.00,0.000000,0.000000,0.000000,0.050000,0.00000,0.000000,0.000000,0.016667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.0000,0.000000,0.033333,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,

In [103]:
kl_mall = kl_grouped[["Neighborhoods","Restaurant"]]

In [104]:
kl_mall.head()

,Neighborhoods,Restaurant
0,ANG MO KIO,0.250000
1,BEDOK,0.383333
2,BISHAN,0.255814
3,BUKIT BATOK,0.269231
4,BUKIT MERAH,0.538462


In [116]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 1, 1, 0, 2, 0, 2], dtype=int32)

In [117]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [118]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,ANG MO KIO,0.250000,0
1,BEDOK,0.383333,2
2,BISHAN,0.255814,0
3,BUKIT BATOK,0.269231,0
4,BUKIT MERAH,0.538462,1


In [119]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(df_data_0.set_index("Town"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(25, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,ANG MO KIO,0.250000,0,1.369972,103.849588
1,BEDOK,0.383333,2,1.324011,103.930172
2,BISHAN,0.255814,0,1.351042,103.849930
3,BUKIT BATOK,0.269231,0,1.348506,103.749222
4,BUKIT MERAH,0.538462,1,1.289642,103.816798


In [90]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(25, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,ANG MO KIO,0.000000,0,1.369972,103.849588
22,TOA PAYOH,0.000000,0,1.332330,103.847425
20,SERANGOON,0.000000,0,1.349787,103.873635
18,SEMBAWANG,0.000000,0,1.449080,103.820058
17,QUEENSTOWN,0.000000,0,1.294835,103.805902
16,PUNGGOL,0.000000,0,1.405170,103.902356
14,MARINE PARADE,0.000000,0,1.308410,103.888814
13,KALLANG/WHAMPOA,0.000000,0,1.311478,103.871351
23,WOODLANDS,0.000000,0,1.436945,103.786516
11,JURONG EAST,0.000000,0,1.333143,103.742329


In [120]:
import folium
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [121]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,ANG MO KIO,0.250000,0,1.369972,103.849588
2,BISHAN,0.255814,0,1.351042,103.849930
3,BUKIT BATOK,0.269231,0,1.348506,103.749222
6,CENTRAL,0.270000,0,1.288155,103.846718
8,CLEMENTI,0.287879,0,1.315070,103.765246
10,HOUGANG,0.230769,0,1.371331,103.892544
13,KALLANG/WHAMPOA,0.222222,0,1.311478,103.871351
15,PASIR RIS,0.212121,0,1.373191,103.949353
17,QUEENSTOWN,0.250000,0,1.294835,103.805902
18,SEMBAWANG,0.235294,0,1.449080,103.820058


In [122]:

kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
4,BUKIT MERAH,0.538462,1,1.289642,103.816798
5,BUKIT PANJANG,0.444444,1,1.276068,103.791904
12,JURONG WEST,0.426667,1,1.338556,103.705828


In [123]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
1,BEDOK,0.383333,2,1.324011,103.930172
7,CHOA CHU KANG,0.400000,2,1.385385,103.744337
9,GEYLANG,0.357143,2,1.316367,103.882772
11,JURONG EAST,0.323944,2,1.333143,103.742329
14,MARINE PARADE,0.321429,2,1.308410,103.888814
16,PUNGGOL,0.304348,2,1.405170,103.902356
20,SERANGOON,0.317073,2,1.349787,103.873635
21,TAMPINES,0.325581,2,1.354430,103.942760
23,WOODLANDS,0.347826,2,1.436945,103.786516
24,YISHUN,0.346939,2,1.429548,103.835033
